#### pip install libraries

In [ ]:
!pip install colorlog
!pip install mysqlclient==2.1.1
!pip install pandas
!pip install psycopg2
!pip install PyJWT==2.6.0
!pip install pymongo
!pip install requests
!pip install git+https://github.com/JaneliaSciComp/jrc_common.git@9958f27a98fc73d23881ef038e6ee387fe715954

#### Import required libraries

In [ ]:
import argparse
import datetime
from operator import attrgetter
import sys
import MySQLdb
import pandas as pd
import jrc_common.jrc_common as JRC

#### SQL statements

In [ ]:
# Database
READ = {"MAIN": "SELECT loh.stock_name,sf.Project AS project,sf.Project_SubCat AS subcat"
                + ",YEAR(date_filled) AS year,COUNT(1) AS cnt FROM "
                + "FlyStore_line_order_history_vw loh  JOIN StockFinder sf ON "
                + "(sf.Stock_Name=loh.stock_name AND sf.Project IS NOT NULL) WHERE "
                + "loh.stock_name IS NOT NULL AND loh.stock_name != 'KEEP EMPTY' "
                + "AND date_filled IS NOT NULL GROUP BY 1,2,3,4"
       }
DB = {}

#### Functions

In [ ]:
def terminate_program(msg=None):
    """ Log an optional error to output, close files, and exit
        Keyword arguments:
          err: error message
        Returns:
          None
    """
    if msg:
        LOGGER.critical(msg)
    sys.exit(-1 if msg else 0)


def initialize_program():
    """ Initialize program
        Keyword arguments:
          None
        Returns:
          None
    """
    try:
        data = JRC.get_config("databases")
    except Exception as err: # pylint: disable=broad-exception-caught)
        terminate_program(err)
    dbo = attrgetter("flyboy.prod.read")(data)
    LOGGER.info("Connecting to %s prod on %s as %s", dbo.name, dbo.host, dbo.user)
    try:
        DB['flyboy'] = JRC.connect_database(dbo)
    except MySQLdb.Error as err:
        terminate_program(JRC.sql_error(err))

#### Initialize the program by getting parameters and opening a connection to FlyBoy

In [ ]:
PARSER = argparse.ArgumentParser(description="FlyStore order report")
PARSER.add_argument('--file', dest='FILE', action='store',
                    default='flystore_order_report.xlsx', help='Output file')
PARSER.add_argument('--verbose', dest='VERBOSE', action='store_true',
                    default=True, help='Flag, Chatty')
PARSER.add_argument('--debug', dest='DEBUG', action='store_true',
                    default=False, help='Flag, Very chatty')
ARG = PARSER.parse_args('')
LOGGER = JRC.setup_logging(ARG)
initialize_program()

#### Get orders from FlyBoy

In [ ]:
    LOGGER.info("Fetching orders")
    try:
        DB['flyboy']['cursor'].execute(READ['MAIN'])
        rows = DB['flyboy']['cursor'].fetchall()
    except MySQLdb.Error as err:
        terminate_program(JRC.sql_error(err))
    LOGGER.info("Found %d orders", len(rows))

#### Create a dict keyed by stock name with the value a dict containing project, subcat, and yearly order counts

In [ ]:
    stock = {}
    minyear = maxyear = datetime.datetime.now().year
    for row in rows:
        if row['year'] < minyear:
            minyear = row['year']
        if row['stock_name'] not in stock:
            stock[row['stock_name']] = {'project': row['project'], 'subcat': row['subcat']}
        stock[row['stock_name']][row['year']] = row['cnt']
    LOGGER.info("Found %d stocks", len(stock))

#### Create an array of dicts containing stock name, project, subcat, yearly counts, and total count

In [ ]:
    prow = []
    for stk, row in stock.items():
        payload = {'Stock': stk, 'Project': row['project'], 'SubCat': row['subcat']}
        total = 0
        for year in range(minyear, maxyear+1):
            if year in row:
                payload[year] = row[year]
                total += payload[year]
            else:
                payload[year] = 0
        payload['Total'] = total
        prow.append(payload)

#### Create a dataframe and output it as an Excel spreadsheet

In [ ]:
    pdf = pd.DataFrame(prow)
    LOGGER.info("Will output %d rows for years %d-%d", pdf.shape[0], minyear, maxyear)
    pdf.to_excel(ARG.FILE, index=False)
    print(f"Wrote report to {ARG.FILE}")